In [1]:
print(__doc__)

import numpy as np
from sklearn.svm import SVR

import pandas as pd

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

Automatically created module for IPython interactive environment


/Users/OptimusPrime/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/Users/OptimusPrime/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#dep

In [2]:
from svc_firstRIs_escape_FTD_hh2dof import train_test_svc_ftd
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

trainingdata_size = 10000
total_energy = 0.17
time = 30

# datapath = '../data/hh2dof_firstRIs/fixed_trainingdata_size/'
# datapath = '../data/hh2dof_firstRIs/newfeatures_added/'
# datapath = '../data/hh2dof_firstRIs_sos_y-0.25/fixed_trainingdata_size/'
# datapath = '../data/hh2dof_firstRIs_sos_y0.5/fixed_trainingdata_size/'


# data = pd.read_csv(datapath + 'hh_escape_samples%d'%(trainingdata_size) \
#                    + '_E%.3f'%(total_energy) \
#                    + '_T%.3f'%(time) + '.txt', \
#                    sep = " ", header = None, \
#                    names = ["x", "y", "p_x", "p_y","Exit channel"])


# datapath = '../data/hh2dof_firstRIs/svcr/'
datapath = '../data/hh2dof_firstRIs_sos_y-0.25/svcr/'
# datapath = '../data/hh2dof_firstRIs_sos_y0.5/svcr/'
data = pd.read_csv(datapath + 'hh_escape_samples%d'%(trainingdata_size) \
                   + '_E%.3f'%(total_energy) \
                   + '_T%.3f'%(time) + '.txt', \
                   sep = " ", header = None, \
                   names = ["x", "y", "p_x", "p_y", "TE", "LD", "Exit channel"])

data = data.dropna()

X = data.drop(["y", "p_y", "TE", "Exit channel"], axis=1)
# X = data.drop(["Exit channel"], axis=1)
y = data["Exit channel"].astype(np.int)
        
# train_test_svc_ftd(total_energy, time, trainingdata_size, data)
X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size=0.2, random_state=0)



#%% Kernel parameter optimization using grid search with cross-validation

# Set the parameters by cross-validation
# C_range = np.logspace(-3, 10, 14)
# gamma_range = np.logspace(-5, 5, 11)
# C_range  = [1, 10, 1e2]
# gamma_range = [1e-1, 1, 10]
# C_range  = [10, 100, 1000]
# gamma_range = [1, 10, 100]
C_range  = [100, 1000, 1e4, 1e5]
gamma_range = [10, 100, 1e3, 1e4]
tuned_parameters = [{'kernel': ['rbf'], \
                     'gamma': gamma_range, \
                     'C': C_range}]

#tuned_parameters = [{'kernel': ['rbf'], \
#                     'gamma': [1e2, 10, 1, 1e-1, 1e-2, 1e-3, 1e-4], \
#                     'C': [1, 10, 1e2, 1e3, 1e4, 1e5]}]
#scoring_metric = 'accuracy'
scoring_metric = 'f1_weighted'

clf = GridSearchCV(SVC(), tuned_parameters, scoring = scoring_metric, cv = 5)
#clf = GridSearchCV(SVC(), tuned_parameters, scoring = 'f1_weighted', cv = 5)

clf.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (clf.best_params_, clf.best_score_))

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()

y_true, y_pred = y_test, clf.predict(X_test)
report = classification_report(y_true, y_pred, output_dict=True)
print(classification_report(y_true, y_pred, output_dict=True))
print()

None


/Users/OptimusPrime/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/Users/OptimusPrime/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:670: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check

The best parameters are {'C': 100, 'gamma': 10, 'kernel': 'rbf'} with a score of 1.00
Best parameters set found on development set:

{'C': 100, 'gamma': 10, 'kernel': 'rbf'}

Grid scores on development set:

0.999 (+/-0.002) for {'C': 100, 'gamma': 10, 'kernel': 'rbf'}
0.994 (+/-0.005) for {'C': 100, 'gamma': 100, 'kernel': 'rbf'}
0.965 (+/-0.004) for {'C': 100, 'gamma': 1000.0, 'kernel': 'rbf'}
0.955 (+/-0.003) for {'C': 100, 'gamma': 10000.0, 'kernel': 'rbf'}
0.999 (+/-0.002) for {'C': 1000, 'gamma': 10, 'kernel': 'rbf'}
0.994 (+/-0.005) for {'C': 1000, 'gamma': 100, 'kernel': 'rbf'}
0.965 (+/-0.004) for {'C': 1000, 'gamma': 1000.0, 'kernel': 'rbf'}
0.955 (+/-0.003) for {'C': 1000, 'gamma': 10000.0, 'kernel': 'rbf'}
0.999 (+/-0.002) for {'C': 10000.0, 'gamma': 10, 'kernel': 'rbf'}
0.994 (+/-0.005) for {'C': 10000.0, 'gamma': 100, 'kernel': 'rbf'}
0.965 (+/-0.004) for {'C': 10000.0, 'gamma': 1000.0, 'kernel': 'rbf'}
0.955 (+/-0.003) for {'C': 10000.0, 'gamma': 10000.0, 'kernel': 'rbf'

In [4]:
import joblib
import pandas as pd

df = pd.DataFrame(report).transpose()
df.to_csv('hh_classificationreport_samples%d'%(trainingdata_size) \
          + '_E%.3f'%(total_energy) \
          + '_T%.3f'%(time) + '.csv') 

# save the model to disk
joblib.dump(clf, 'hh_svc_samples%d'%(trainingdata_size) \
            + '_E%.3f'%(total_energy) \
            + '_T%.3f'%(time) + '.sav')

['hh_svc_samples10000_E0.170_T30.000.sav']

In [9]:
import henonheiles
import importlib
importlib.reload(henonheiles)
import henonheiles as HH2dof

params = [1.0, 1.0, 1.0, 1.0, 1.0]
y_constant = 0.0

res = 300
xMax_at_yconstant = np.sqrt((total_energy - \
                     (0.5*params[3]**2*y_constant**2 - \
                      (params[4]/3)*y_constant**3))/(0.5*params[2]**2 + y_constant))
xMin_at_yconstant = -xMax_at_yconstant

pxMax_at_yconstant = np.sqrt(2*params[0]*(total_energy - HH2dof.potential_energy(0,y_constant, params[2:])))
pxMin_at_yconstant = -pxMax_at_yconstant


px_at_yconstant = lambda x: np.sqrt(2*params[0]*(total_energy - \
                                    (0.5*params[2]**2*x**2 + 0.5*params[3]**2*y_constant**2 - \
                                     (params[4]/3.0)*y_constant**3 + x**2*y_constant) ))


xGrid_boundary = np.linspace(xMin_at_yconstant + 1e-10, xMax_at_yconstant - 1e-10, 401, endpoint = True)
pxGrid_boundary = px_at_yconstant(xGrid_boundary)
pxGrid_boundary[0] = 0
pxGrid_boundary[-1] = 0

xGrid_boundary = np.append(xGrid_boundary, np.flip(xGrid_boundary))
pxGrid_boundary = np.append(pxGrid_boundary, -np.flip(pxGrid_boundary))


xMesh, pxMesh = np.meshgrid(np.linspace(xMin_at_yconstant + 1e-6, xMax_at_yconstant - 1e-6, res), \
                           np.linspace(pxMin_at_yconstant + 1e-6, pxMax_at_yconstant - 1e-6, res))

        
X_grid = np.column_stack([xMesh.ravel(), pxMesh.ravel()])
# LD_predict = svr_rbf_fit.predict(X_grid)
# X_grid = np.column_stack([X_grid, LD_predict])
# scaler = StandardScaler()
# X_grid = scaler.fit_transform(X_grid)

# decision_grid = clf_model.best_estimator_.decision_function(X_grid)
decision_grid = clf.best_estimator_.predict(X_grid)



ValueError: X.shape[1] = 2 should be equal to 3, the number of features at training time

In [ ]:
import joblib

svr_rbf = SVR(kernel='rbf', C = 100, gamma = 10, epsilon = 0.05)

svr_rbf_fit = svr_rbf.fit(X_train, y_train)

joblib.dump(svr_rbf_fit, 'hh_svr_samples%d'%(trainingdata_size) \
            + '_E%.3f'%(total_energy) + '_T%.3f'%(time) + '.sav')


In [ ]:


fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize=(20, 10), sharey=True)
sc0 = axes[0].scatter(X_train["x"], X_train["p_x"], c = y_train, s = 10)
# axes[1].scatter(X_test["x"], X_test["p_x"], c = svr_rbf_fit.predict(X_test), s = 10)
sc1 = axes[1].scatter(X_train["x"], X_train["p_x"], c = svr_rbf_fit.predict(X_train), s = 10)
fig.colorbar(sc0, ax = axes[0])
fig.colorbar(sc1, ax = axes[1])
axes[0].set_ylabel(r'$p_x$', labelpad = 5, rotation = 0, fontsize = 30)
axes[0].set_xlabel(r'$x$', labelpad = 0, fontsize = 30)
axes[1].set_xlabel(r'$x$', labelpad = 0, fontsize = 30)

plt.subplots_adjust(wspace=0, hspace=0)
fig.savefig('temp.png', dpi = 300, bbox_inches = 'tight')


## Regression model of the LD values

In [ ]:

# total_energy = 0.19
# time = 30
# trainingdata_size = 10000
# datapath = '../data/hh2dof_firstRIs/newfeatures_added/'
# data = pd.read_csv(datapath + 'hh_escape_samples%d'%(trainingdata_size) \
#                    + '_E%.3f'%(total_energy) \
#                    + '_T%.3f'%(time) + '.txt', \
#                    sep = " ", header = None, \
#                    names = ["x", "y", "p_x", "p_y", "TE", "LD", "Exit channel"])

#print(data.shape,'\n',data.head)

data = data.dropna()
print('Samples on the energy surface:%d'%(data.shape[0]),'\n',data.head)


X = data.drop(["y", "p_y", "TE", "LD", "Exit channel"], axis=1)
# X = data.drop(["TE", "LD", "Exit channel"], axis=1)
y = data["LD"]

X_train, X_test, y_train, y_test = train_test_split(X, y, \
                                                    test_size=0.2, random_state=0)

